In [21]:
%maven junit:junit:4.13.2
%load djl_imports
import ai.djl.training.loss.L2Loss;

NDManager manager = NDManager.newBaseManager();
    
public void print(Object o) {
    System.out.println(o);
}

In [22]:
public NDArray add10(NDArray parameter) {
    return parameter.add(10);
}

In [23]:
try (NDManager manager = NDManager.newBaseManager()) {
    // Initialize a learnable parameter and expected value
    NDArray parameter = manager.create(0.0f); // Start with an initial guess
    NDArray expected = manager.create(new float[]{100.0f});
    // Set gradients for the input param
    parameter.setRequiresGradient(true);

    // Training loop
    float learningRate = 0.5f;
    for (int i = 0; i < 10; i++) {
        // Create a new gradient collector - these need to made each training iteration
        try (GradientCollector gc = Engine.getInstance().newGradientCollector()) {
            // 
            NDArray result = add10(parameter);
            Loss lossFunction = Loss.l2Loss();
            NDArray loss = lossFunction.evaluate(new NDList(expected), new NDList(result));

            gc.backward(loss); // Calculate gradients
            print("Iteration: " + i + " Loss: " + loss.getFloat());

        }
        // Update the parameter (gradient descent)
        parameter.subi(parameter.getGradient().mul(learningRate)); 
        // Clear gradients otherwise memory creeps
        parameter.getGradient().close();
    }

    // We need the parameter to be close to `90` in order 
    // for the result to match the expected value of `100`
    print("Learned parameter: " + parameter.getFloat());
}


Iteration: 0 Loss: 4050.0
Iteration: 1 Loss: 1012.5
Iteration: 2 Loss: 253.125
Iteration: 3 Loss: 63.28125
Iteration: 4 Loss: 15.8203125
Iteration: 5 Loss: 3.9550781
Iteration: 6 Loss: 0.98876953
Iteration: 7 Loss: 0.24719238
Iteration: 8 Loss: 0.061798096
Iteration: 9 Loss: 0.015449524
Learned parameter: 89.91211
